In [1]:
import pandas as pd
import numpy as np
import string
import re

from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
path = './../data/'

In [3]:
data = pd.read_csv(f"{path}AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt", 
                   encoding='cp1252', 
                   sep="|", 
                   header=None,
                   na_filter=False)
data.columns = ["abbrev", "sense", "represntaion", "start_pos", "end_pos", "section_info", "sample"]

In [4]:
data.head()

,abbrev,sense,represntaion,start_pos,end_pos,section_info,sample
0,AB,abortion,AB.,231,233,,_%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...
1,AB,abortion,AB.,249,251,,She is now bleeding quite heavily. Ultrasound ...
2,AB,abortion,AB,223,224,PAST OB HISTORY,ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...
3,AB,abortion,AB.,194,196,HISTORY OF THE PRESENT ILLNESS,She had a pelvic ultrasound at Park Nicollet o...
4,AB,abortion,AB,114,115,PAST OB-GYN HISTORY,"On _%#MMDD2007#%_, normal anatomy with anterio..."


In [5]:
unique_abbrev = np.unique(data.abbrev)

In [6]:
empty_dict = dict.fromkeys(['abbrev','number_sense'])

In [7]:
abbrev_freq = pd.DataFrame(columns=["abbrev", "number_sense", "sense", "freq", "percentage"])
#abbrev_freq_dict = dict.fromkeys(['abbrev','number_sense'])

count_list = []
for abbrev in unique_abbrev:
    piece = data.loc[data.abbrev == abbrev]
    senses = np.unique(piece.sense)
    count = len(np.unique(piece.sense))
    for sense in senses:
        count_sense = piece.loc[piece.sense == sense].shape[0]
        percentage = count_sense/piece.shape[0]
        new = pd.DataFrame({"abbrev" : [abbrev],
                            "number_sense" : [count],
                            "sense" : [sense],
                            "freq" : [count_sense],
                            "percentage" : [percentage]})
        
        abbrev_freq = pd.concat([abbrev_freq, new])
    #count_list.append(count)
    
#abbrev_freq = pd.DataFrame({"abbrev" : unique_abbrev, "number_sense" : count_list})

In [8]:
np.unique(abbrev_freq.abbrev)

array(['AB', 'AC', 'ALD', 'AMA', 'ASA', 'AV', 'AVR', 'BAL', 'BK', 'BM',
       'BMP', 'C&S', 'C3', 'C4', 'CA', 'CDI', 'CEA', 'CR', 'CTA', 'CVA',
       'CVP', 'CVS', 'DC', 'DIP', 'DM', 'DT', 'EC', 'ER', 'ES', 'ET',
       'FISH', 'FSH', 'GT', 'IA', 'IB', 'IM', 'IR', 'IT', 'ITP', 'IVF',
       'LA', 'LE', 'MOM', 'MP', 'MR', 'MS', 'MSSA', 'NA', 'NAD', 'NP',
       'OP', 'OR', 'OTC', 'PA', 'PAC', 'PCP', 'PD', 'PDA', 'PE', 'PM',
       'PR', 'PT', 'RA', 'RT', 'SA', 'SBP', 'SMA', 'SS', 'T1', 'T2', 'T3',
       'T4', 'US', 'VAD', 'VBG'], dtype=object)

In [9]:
abbrev_freq.loc[abbrev_freq.abbrev == "AMA"]

,abbrev,number_sense,sense,freq,percentage
0,AMA,3,advanced maternal age,31,0.062
0,AMA,3,against medical advice,444,0.888
0,AMA,3,antimitochondrial antibody,25,0.050


In [10]:
# feature extraction
# if select AC

In [11]:
def derive_features(abbrev, window_size, replace = True):  # replace = True, replace numbers with zero
    samples = data.loc[data.abbrev == abbrev, ]
    sample_num = 1
    
    original_features = pd.DataFrame(columns=["id", "features", "sense"])
    direction_features = pd.DataFrame(columns=["id", "features", "sense"])
    direction_num_features = pd.DataFrame(columns=["id", "features", "sense"])
    
    for i in range(samples.shape[0]):  # for each data point
        sentence_num = 1
        #target_word = samples.iloc[i, 2]  # should avoid using this, as word tokenization would split AC. to AC ., etc.
        #target_word = abbrev
        
        text = samples.iloc[i, 6]
        id = i+1  # this is the row number of selected abbreviation
        sense = samples.iloc[i, 1]
        
        target_word = samples.iloc[i, 2]
        sentence_length = 0
        start_pos = int(samples.iloc[i, 3])
        end_pos = int(samples.iloc[i, 4])
        
        target_word = text[start_pos : end_pos+1]
        detect = True
        #print(target_word)
        # sentence boundary
        # one sample can have multiple abbreviations in different sentences.
        # before sentence boundary, replace all the numbers with 0 
        #print(text)
        
        
        #if replace:
        #    text = re.sub("\d+", "0", text)
        #    text = re.sub("\d+\.\d+", "0", text)

        sents = sent_tokenize(text)
        exclude = set(string.punctuation)
        #s = ''.join(ch for ch in s if ch not in exclude)
        for sent in sents:  # for each sentence
            #words = [token.lower() for token in word_tokenize(sent)] # word tokenization
            # remove punctuations from the words list
            sentence_length += (len(sent)+1)
            words = [word for word in word_tokenize(sent) if word not in string.punctuation]
            
#             print(id, "____________________")
#             print(sent)
#             print(words)
#             print(target_word)
#             print(sentence_length, end_pos)
            if sentence_length >= end_pos and (abbrev in words or target_word in words) and detect:
                #print(words)
                #print(sent)
                detect = False
                left_features = []
                right_features = []
                
                left_features_direction = []
                right_features_direction = []
                
                left_features_direction_num = []
                right_features_direction_num = []
                
                index = words.index(abbrev)
                # find the targeted word
                # 1. See if the window-size exceeds the front and back limit
                    # If yes, start from the zero-th element, towards right till find the target (features on the left)
                            # start from the max-th element, towards left till find the target (features on the right)
                    # If no, start from the (index - 5)-th element, towards right till find the target
                           # start from the (index - 5)-th element, towards left till find the target
                        
                # extract features on the left
                if index - window_size < 0:
                    j = 0
                    starting = index
                    #while words[j] != target_word:
                    while words[j] != abbrev:
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        if replace:
                            words[j] = re.sub("\d+", "0", words[j])
                            words[j] = re.sub("\d+\.\d+", "0", words[j])
                        # remove punctuation from the word, meant to fix problem in word tokenization
                        # but may cause problem, e.g. p.r.n --> prn
                        # may remove later
                        
                        left_features.append(words[j].lower())
                        left_features_direction.append("L-" + words[j].lower())
                        left_features_direction_num.append("L" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting -= 1
                else:
                    j = index-window_size
                    starting = 0
                    for k in range(window_size):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        if replace:
                            words[j] = re.sub("\d+", "0", words[j])
                            words[j] = re.sub("\d+\.\d+", "0", words[j])
                        left_features.append(words[j].lower())
                        left_features_direction.append("L-" + words[j].lower())
                        left_features_direction_num.append("L" + str(window_size-starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                
                # extract feature on the right
                if index + window_size >= len(words):
                    #j = len(words)-1
                    j = index+1
                    starting = 1
                    while j != len(words):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        if replace:
                            words[j] = re.sub("\d+", "0", words[j])
                            words[j] = re.sub("\d+\.\d+", "0", words[j])
                        right_features.append(words[j].lower())
                        right_features_direction.append("R-" + words[j].lower())
                        right_features_direction_num.append("R" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                else: 
                    j = index+1
                    starting = 1
                    for k in range(window_size):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        if replace:
                            words[j] = re.sub("\d+", "0", words[j])
                            words[j] = re.sub("\d+\.\d+", "0", words[j])
                        right_features.append(words[j].lower())
                        right_features_direction.append("R-" + words[j].lower())
                        right_features_direction_num.append("R" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                   
                left_features_str = " ".join(left_features)
                right_features_str = " ".join(right_features)
                features = left_features_str + " " + right_features_str
                
#                 print(features)
                
                left_features_direction_str = " ".join(left_features_direction)
                right_features_direction_str = " ".join(right_features_direction)
                features_direction = left_features_direction_str + " " + right_features_direction_str
                
                left_features_direction_num_str = " ".join(left_features_direction_num)
                right_features_direction_num_str = " ".join(right_features_direction_num)
                features_direction_num = left_features_direction_num_str + " " + right_features_direction_num_str
                #print(features_direction_num)
                
#                 if replace:
#                     features = re.sub("\d+", "0", features)
#                     features = re.sub("\d+\.\d+", "0", features)
                    
#                     features_direction = re.sub("\d+", "0", features_direction)
#                     features_direction = re.sub("\d+\.\d+", "0", features_direction)
                    
#                     features_direction_num = features_direction_num+re.sub("\d+", "0", features_direction_num)
#                     features_direction_num = features_direction_num+re.sub("\d+\.\d+", "0", features_direction_num)
#                     print(features_direction_num)
                original_features = pd.concat([original_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features], 
                                                                               "sense" : [sense]})])
                
                direction_features = pd.concat([direction_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features_direction], 
                                                                               "sense" : [sense]})])
                
                direction_num_features = pd.concat([direction_num_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features_direction_num], 
                                                                               "sense" : [sense]})])
            sentence_num += 1
        sample_num += 1
    return([original_features, direction_features, direction_num_features])
                

In [12]:
ar1, br1, cr1 = derive_features('AMA', 5, replace=True)

# ONE-HOT

In [13]:
def one_hot_features(abbrev, window_size, replace = True):
    ar1, br1, cr1 = derive_features(abbrev, window_size, replace = True)
    one_hot_vector=[]
    for k in cr1['features'].values:
        features_direction_num= [item for item in k.split(' ') if item !='']
        one_hot_vector=one_hot_vector+features_direction_num
    one_hot_vector=sorted(list(set(one_hot_vector)))
    char_to_int = dict((c, i) for i, c in enumerate(one_hot_vector))
    
    onehot_encoded = list()
    for k in cr1['features'].values:
        features_direction_num= [item for item in k.split(' ') if item !='']
        integer_encoded = [char_to_int[char] for char in features_direction_num]
        # one hot encode

        letter = [0 for _ in range(len(char_to_int))]
        for value in integer_encoded:
            letter[value] = 1
        onehot_encoded.append(letter)
    onehot_encoded=np.array(onehot_encoded)
    return onehot_encoded,one_hot_vector

In [14]:
features = {}
names = {}

for i in range(1,6):
    features[i],names[i]=one_hot_features('AMA', i, replace=True)#not use ald as they have number of sense=1


In [15]:
# fea_5,name_5=one_hot_features('AMA', 5, replace=True)#not use ald as they have number of sense=1
# fea_4,name_4=one_hot_features('AMA', 4, replace=True)
# fea_3,name_3=one_hot_features('AMA', 3, replace=True)
# fea_2,name_2=one_hot_features('AMA', 2, replace=True)
# fea_1,name_1=one_hot_features('AMA', 1, replace=True)

# machine learning to decide window size

In [16]:
from sklearn.model_selection import train_test_split

y=cr1['sense'].values

train_X = {} 
test_X = {}
train_y = {}
test_y = {}

for i in range(1,6):
    train_X[i],test_X[i],train_y[i],test_y[i] = train_test_split(features[i], y, test_size=0.2, random_state=13, shuffle=True, stratify=y)
    

In [17]:
# train_X_5, test_X_5, train_y, test_y = train_test_split(fea_5, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)
# train_X_4, test_X_4, train_y, test_y = train_test_split(fea_4, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)
# train_X_3, test_X_3, train_y, test_y = train_test_split(fea_3, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)
# train_X_2, test_X_2, train_y, test_y = train_test_split(fea_2, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)
# train_X_1, test_X_1, train_y, test_y = train_test_split(fea_1, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

def five_fold_CV(clf, params, dx, dy):
    cv_model = RandomizedSearchCV(clf, params, scoring='f1_weighted', n_jobs=-1, 
                                  cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=13), 
                                  verbose=1, n_iter=50, refit=True)
    
    cv_model.fit(dx, dy)
    
    return cv_model.best_estimator_

In [19]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

def logistic_regression_pred(X_train, Y_train, X_test):
    lr = LogisticRegression()
    tuned_parameters = {'max_iter': [10, 50, 100, 200, 500, 750, 1000], 
                    'tol': [0.0001, 0.001, 0.01, 0.1],
                    'C': [0.01, 0.1, 1.0, 5.0, 10.0, 25.0, 50.0, 100.0],
                    'solver': ['lbfgs', 'liblinear', 'newton-cg'], 
                    'class_weight': [None, 'balanced']}
    
    best_lr_model = five_fold_CV(lr, tuned_parameters, X_train, Y_train)
    lr_y_test = best_lr_model.predict(X_test)
    return lr_y_test

def svm_pred(X_train, Y_train, X_test):
    svm = SVC()
    tuned_parameters = {"C": [0.001, 0.01, 0.1, 2, 8, 32, 64, 128, 512, 1024, 2048],
                    'gamma':['scale', 'auto'],
                    'probability':[True], 
                    'tol': [0.1, 0.01, 0.001, 0.0001]}
    best_svm_model = five_fold_CV(svm, tuned_parameters, X_train, Y_train)
    svm_y_test = best_svm_model.predict(X_test)
    return svm_y_test

def knn_pred(X_train, Y_train, X_test):
    neigh = KNeighborsClassifier()
    tuned_parameters = dict(n_neighbors=range(1, 30), weights=['uniform', 'distance'])
    best_knn_model = five_fold_CV(neigh, tuned_parameters, X_train, Y_train)
    neigh_y_pred = best_knn_model.predict(X_test)
    return neigh_y_pred

def randomforest_pred(X_train, Y_train, X_test):
    rf = RandomForestClassifier()
    
    tuned_parameters = {'n_estimators':[50, 100, 250,  500, 750, 1000, 1250, 1500, 2000], 
                    'criterion':['gini', 'entropy'], 
                    'max_features':['log2', 'auto', None], 
                    'min_samples_split':[2, 3, 4], 
                    'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 32, None], 
                    'min_samples_leaf':[1, 2], 
                    'max_leaf_nodes': [None, 5, 10],
                    'min_impurity_decrease':[0.1, 0.01, 0.001, 0.0001, 0.00001],
                    'bootstrap': [True, False],
                    'class_weight': [None, 'balanced', 'balanced_subsample']}
    best_rf_model = five_fold_CV(rf, tuned_parameters, X_train, Y_train)

    forest_y_pred = best_rf_model.predict(X_test)
    return forest_y_pred 

def classification_metrics(Y_pred, Y_true):
    #TODO: Calculate the above mentioned metrics
    #NOTE: It is important to provide the output in the same order
    precision = precision_score(Y_pred, Y_true, average='micro')
    recall = recall_score(Y_pred, Y_true, average='micro')
    f1score = f1_score(Y_pred, Y_true, average='micro')
    return precision,recall,f1score

def display_metrics(classifierName, Y_pred, Y_true):
    print("______________________________________________")
    print(("Classifier: "+classifierName))
    precision, recall, f1score = classification_metrics(Y_pred,Y_true)
    print(("Precision: "+str(precision)))
    print(("Recall: "+str(recall)))
    print(("F1-score: "+str(f1score)))
    print("______________________________________________")
    print("")

In [20]:
# from sklearn.metrics import f1_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import accuracy_score
# def report_result(y_test,predicted_target):
# #     print('precision',precision_score(y_test, predicted_target,average='macro'))
# #     print('f1',f1_score(y_test, predicted_target, average='macro'))
# #     print('recall',recall_score(y_test, predicted_target, average='macro'))
#     print('recall',accuracy_score(y_test, predicted_target))#recall should be accuracy if in WSD problem
    
    

In [21]:
# LR - window size (1,5)
for i in range(1,6):
    display_metrics("Logistic Regression",logistic_regression_pred(train_X[i],train_y[i],test_X[i]),test_y[i])

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: Logistic Regression
Precision: 0.98
Recall: 0.98
F1-score: 0.98
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: Logistic Regression
Precision: 0.96
Recall: 0.96
F1-score: 0.96
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: Logistic Regression
Precision: 0.96
Recall: 0.96
F1-score: 0.96
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: Logistic Regression
Precision: 0.95
Recall: 0.95
F1-score: 0.9500000000000001
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 2

In [22]:
for i in range(1,6):
    display_metrics("SVM",svm_pred(train_X[i],train_y[i],test_X[i]),test_y[i])

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: SVM
Precision: 0.91
Recall: 0.91
F1-score: 0.91
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: SVM
Precision: 0.95
Recall: 0.95
F1-score: 0.9500000000000001
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: SVM
Precision: 0.95
Recall: 0.95
F1-score: 0.9500000000000001
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: SVM
Precision: 0.94
Recall: 0.94
F1-score: 0.94
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
__________________________________________

In [23]:
for i in range(1,6):
    display_metrics("K Nearest Neighbor",knn_pred(train_X[i],train_y[i],test_X[i]),test_y[i])

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: K Nearest Neighbor
Precision: 0.91
Recall: 0.91
F1-score: 0.91
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: K Nearest Neighbor
Precision: 0.83
Recall: 0.83
F1-score: 0.83
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: K Nearest Neighbor
Precision: 0.66
Recall: 0.66
F1-score: 0.66
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
______________________________________________
Classifier: K Nearest Neighbor
Precision: 0.79
Recall: 0.79
F1-score: 0.79
______________________________________________

Fitting 5 folds for each of 50 candidates, totalling 250 fits
__________

In [25]:
# for i in range(1,6):
#     display_metrics("RF",randomforest_pred(train_X[i],train_y[i],test_X[i]),test_y[1])